In [1]:
from dask.distributed import Client
from dask_yarn import YarnCluster

cluster = YarnCluster(worker_memory='16GiB')
client = Client(cluster)

/opt/conda/miniconda3/lib/python3.10/site-packages/dask_yarn/core.py:16: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import (
/opt/conda/miniconda3/lib/python3.10/site-packages/dask_yarn/core.py:16: FutureWarning: parse_timedelta is deprecated and will be removed in a future release. Please use dask.utils.parse_timedelta instead.
  from distributed.utils import (
23/10/04 21:30:08 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at dask-m.us-west1-c.c.galvanic-ripsaw-376002.internal./10.138.0.30:8032
23/10/04 21:30:08 INFO client.AHSProxy: Connecting to Application History server at dask-m.us-west1-c.c.galvanic-ripsaw-376002.internal./10.138.0.30:10200
23/10/04 21:30:08 INFO skein.Driver: Driver started, listening on 44091
23/10/04 21:30:09 INFO conf.Configuration: resource-types.xml not found
23/10/04 21:30:09 INFO resource.ResourceUtils: Unable to f

In [3]:
%load_ext autoreload
%autoreload 2
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data

In [4]:
df = get_pvdaq_data(sysid=34, year=2011, api_key='DEMO_KEY')

[============================================================] 100.0% ...queries complete in 1.4 seconds       



In [5]:
dh = DataHandler(df)
dh.run_pipeline(power_col='ac_power')

total time: 9.64 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              1.67s
Cleaning                   0.09s
Filtering/Summarizing      7.88s
    Data quality           0.08s
    Clear day detect       0.12s
    Clipping detect        5.31s
    Capacity change detect 2.36s



In [6]:
import dask

@dask.delayed
def run_pipeline(df):
    dh = DataHandler(df)
    dh.run_pipeline(power_col='ac_power')
    return dh.report(return_values=True)

In [7]:
results = []
for _ in range(3):
    results.append(run_pipeline(df))
dask.compute(results)

/opt/conda/miniconda3/lib/python3.10/site-packages/distributed/worker.py:4423: UserWarning: Large object of size 4.55 MiB detected in task graph: 
  (                     SiteID  ac_current  ac_power ... x 16 columns],)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


([{'length': 1.0,
   'capacity': 109.3,
   'sampling': 15,
   'quality score': 0.9835616438356164,
   'clearness score': 0.4986301369863014,
   'inverter clipping': True,
   'clipped fraction': 0.01643835616438356,
   'capacity change': False,
   'data quality warning': False,
   'time shift correction': False,
   'time zone correction': 0},
  {'length': 1.0,
   'capacity': 109.3,
   'sampling': 15,
   'quality score': 0.9835616438356164,
   'clearness score': 0.4986301369863014,
   'inverter clipping': True,
   'clipped fraction': 0.01643835616438356,
   'capacity change': False,
   'data quality warning': False,
   'time shift correction': False,
   'time zone correction': 0},
  {'length': 1.0,
   'capacity': 109.3,
   'sampling': 15,
   'quality score': 0.9835616438356164,
   'clearness score': 0.4986301369863014,
   'inverter clipping': True,
   'clipped fraction': 0.01643835616438356,
   'capacity change': False,
   'data quality warning': False,
   'time shift correction': False,